<a href="https://colab.research.google.com/github/lordtrump/HF/blob/main/Hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install aria2

In [ ]:


# @title Download
url_model = "" # @param {type:"string"}
file_name = "" # @param {type:"string"}
##############

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url_model} -o {file_name}.safetensors

---

In [ ]:

# @title Huggingface Login

# Get your 'HF' token 'WRITE' here: 'https://huggingface.co/settings/tokens'
write_token = ""  # @param {type:"string"}

# If the repository does not exist, it will be automatically created.
repository_name = ""  # @param {type:"string"}
# Make your repository private?
make_private = False  # @param {type:"string"}





# =======================================================================================
import os
import re
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

!git config --global credential.helper store


def sanitize_model_name(name):
    # Remove invalid characters, limit the length, and fix other problems if there are any
    name = re.sub(r"[^\w.-]", "", name)
    name = name[:96]
    name = name.strip("-.")
    name = name.replace("..", ".")
    return name


def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api


def create_repo(api, user, repo_name, repo_type, make_private=False):
    global model_repo

    repo_id = user["name"] + "/" + repo_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type=repo_type, private=make_private)
        print(f"\nRepo '{repo_id}' does not exist, creating the repository.")
    except HfHubHTTPError as e:
        print(f"\nRepo '{repo_id}' already exists, skipping repository creation.")

    model_repo = repo_id
    print(f"Repo '{repo_id}' - link: https://huggingface.co/{repo_id}\n")


user, api = authenticate(write_token)

if repository_name:
    sanitized_model_name = sanitize_model_name(repository_name)
    create_repo(api, user, repo_name=sanitized_model_name, repo_type="model", make_private=make_private)
else:
    create_repo(api, user, repo_name="THE_NAME_WAS_NOT_GIVEN", repo_type="model", make_private=make_private)

In [ ]:


# @title Huggingface Uploads

### Specify the path to the folder/file to upload. | Multiple paths are also supported, just separate them with commas/spaces.
paths_input = "" # @param {type:"string"}
# This is the path of directories that will be created in your repository. (Optional)
path_in_repo = "" # @param {type:"string"}
# Other information.
commit_message = "up" # @param {type:"string"}





# =======================================================================================
import re
from huggingface_hub import HfApi
from pathlib import Path

%store -r

api = HfApi()


if not commit_message:
    commit_message = "no commit message..."


def upload_to_hf_single(path, is_config):
    path_obj = Path(path)
    is_folder = path_obj.is_dir()
    files = path_in_repo + path_obj.parts[-1]

    print(f"Uploading '{files}'")
    print("Please wait... If nothing happens, it doesn't mean that everything is stuck, some files take longer to process.", end=" ")

    if is_folder:
        api.upload_folder(
            folder_path=path,
            path_in_repo=files,
            repo_id=model_repo,
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
        print(f"\rUpload successful! Location: https://huggingface.co/{model_repo}/tree/main/{files}\n---\n")
    else:
        api.upload_file(
            path_or_fileobj=path,
            path_in_repo=files,
            repo_id=model_repo,
            commit_message=commit_message,
        )
        print(f"\rUpload successful! Location: https://huggingface.co/{model_repo}/blob/main/{files}\n---\n")


def upload_to_hf_multi(paths, is_config):
    paths = re.split(r"[,\s]+", paths)

    for path in paths:
        if not path.startswith("/content/"):
            path ="/content/" + path
        upload_to_hf_single(path, is_config)


def upload(paths_input):
    print(f"Uploading to repository '{model_repo}' | https://huggingface.co/{model_repo}\n\n")

    if "," in paths_input or " " in paths_input:
        upload_to_hf_multi(paths_input, False)
    else:
        if not paths_input.startswith("/content/"):
            paths_input = "/content/" + paths_input
        upload_to_hf_single(paths_input, False)


upload(paths_input)

---